In [1]:
import collections
import gc
import re

import cufflinks as cf
import jieba
import matplotlib
import matplotlib.pyplot as plt
import missingno as msno
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud

%matplotlib inline
cf.go_offline()
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',100)
plt.style.use('seaborn')

In [4]:
train_unlabeled = pd.read_csv('./input/train_dataset/nCoV_900k_train_unlabled_utf8.csv', encoding='utf-8')
train_unlabeled.rename(columns = {"微博id": "Weibo_ID",
                                "微博发布时间": "Publish_Time", 
                                "发布人账号": "Account_ID",
                                "微博中文内容": "Chinese_Content",
                                "微博图片": "Pictures",
                                "微博视频": "Videos",
                                "情感倾向": "Labels"},  inplace=True)

train_unlabeled_copy = train_unlabeled.copy()

In [5]:
train_unlabeled_copy

,Weibo_ID,Publish_Time,Account_ID,Chinese_Content,Pictures,Videos
0,4456074364642450,01月01日 23:59,Dear_Christin,2020年第一天，宝宝晚上发烧了。上个月的肺炎经历还历历在目，不觉又开始揪心。目前低烧安稳睡觉中，告诉自己勿要着急。为了缓解情绪刚才又翻了一遍裴医生的文章，作为母亲真不知需要经历多少煎熬才可以...,[],[]
1,4456069114155340,01月01日 23:38,Maggie大美Y,生病发烧一直就没好～就当预示着2020年是个风风火火的好好好运年～新年第一天有肉吃，以后天天有肉吃?,"['https://ww1.sinaimg.cn/orj360/006TMJgaly1gahhj4bmkcj32801o0u0y.jpg', 'https://ww4.sinaimg.cn/t...",[]
2,4456068145061890,01月01日 23:34,折眉几年,害新年第一天就发烧????,[],[]
3,4456064403586410,01月01日 23:20,秃头少女Brushblog,是心动啊，糟糕眼神躲不掉。对你莫名的心跳，竟然停不了对你的迷恋感觉要发烧。@namjoohyuk_official,['https://wx3.sinaimg.cn/orj480/63becc16gy1g7vd28prcbj20hs0a1q4w.jpg'],['https://f.video.weibocdn.com/0009osQVlx07xJgl0r8c010412002xv30E010.mp4?label=mp4_ld&template=3...
4,4456060284446840,01月01日 23:03,Akemi庄蝶,过去的一整年有好好喜欢你。不过今年不可以啦，毕竟，可不能连自己都不要了。跨年给你打电话，听到听筒里远远传来你男朋友和你嬉笑的声音。想起不久前你和你男朋友冷战时，你坚决地说你要和他分手。我站在楼...,[],[]
...,...,...,...,...,...,...
899995,4468336471223800,02月04日 20:04,央视网,【洗手多重要？专家反复强调】湖北省举行中央指导组医疗救治专场新闻发布会，会上蒋荣猛表示，病毒是通过飞沫和接触传播的，飞沫沉降之后更多的会通过手传播，所以一定要把手洗好，今天你洗了几遍手？#最新...,['https://imgaliyuncdn.miaopai.com/images/B8SdJwMY5Pm8oXKt8Q7BjlxYLobcHSmOiYuQwA___oLwx_4.jpg'],['https://miaopai.video.weibocdn.com/000jQ6JJlx07AH0ptoaA010412004JXR0E013.mp4?label=mp4_ld&temp...
899996,4468312186995290,02月04日 18:28,小央视频,#滚蛋吧新冠肺炎#【洗手多重要？专家反复强调】湖北省举行中央指导组医疗救治专场新闻发布会，会上蒋荣猛表示，病毒是通过飞沫和接触传播的，飞沫沉降之后更多的会通过手传播，所以一定要把手洗好，今天你...,['https://imgaliyuncdn.miaopai.com/images/B8SdJwMY5Pm8oXKt8Q7BjlxYLobcHSmOiYuQwA___oLwx_4.jpg'],['https://miaopai.video.weibocdn.com/000jQ6JJlx07AH0ptoaA010412004JXR0E013.mp4?label=mp4_ld&temp...
899997,4468293408183550,02月04日 17:13,浦发北京分行,勤洗手！,['https://wx3.sinaimg.cn/crop.5.0.2385.1342/86a17422ly1gbkaeegvqtj21uq11a4qr.jpg'],['https://fus.cdn.krcom.cn/000MAOaRlx07AGErSxIQ01041200bNGm0E010.mp4?label=mp4_ld&template=640x3...
899998,4468260684595790,02月04日 15:03,大召_医,近距离接触飞沫传播容易感染，所以距离保持远一点，1.5米到2米左右是比较安全的#食堂吃出了考试的感觉#，特殊时期，呼吁大家医院就诊也请保持安全距离。?,['https://ww2.sinaimg.cn/orj360/be0c7538ly1gbkdnj2pd5j21400u0ai8.jpg'],[]
